In [ ]:
%pip install -q pyspark

In [ ]:
import pyspark
import pandas as pd
import numpy as np

# EXTRACT

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.read.format('csv') \
    .option("header", True) \
    .option("inferSchema", True) \
    .load('csv_files/data_lake.csv')

df.show(10)
df=df.toPandas()

In [ ]:
df.head()

Let's drop columns '_c0' and 'Unnamed: 0' as they are uneccesary and provide no useful data.

In [ ]:
df.drop(columns=['_c0', 'Unnamed: 0'], inplace=True)
df

#               TRANSFORM 

# FURNITURE DATAFRAME #

We will create a seperate dataframe that contains all the columns relating to furniture.

In [ ]:
furniture_df = df[['item_id', 'name', 'category', 'price5',\
       'old_price', 'sellable_online', 'link', 'other_colors',\
       'short_description', 'designer', 'depth', 'height', 'width']].copy()
furniture_df.dropna(subset='item_id', inplace=True)
furniture_df.reset_index(inplace=True, drop=True)
furniture_df = furniture_df.rename({'price5': 'price'}, axis=1)
furniture_df

Let's check for any rows that contain null values in the Furniture Dataframe.

In [ ]:
furniture_df[furniture_df.isnull().any(axis=1)]

Notice that the depth, height, and width columns contain some null values for some of the rows.

Since the dimensions of the furniture items are already indicated in the short_description column, we can go ahead and drop the depth, height, and width columns as it is just redundant data.

In [ ]:
furniture_df.drop(columns=['depth', 'height', 'width'], inplace=True)
furniture_df

Let's (again) check for any rows that contain null values in the Furniture Dataframe.

In [ ]:
furniture_df[furniture_df.isnull().any(axis=1)]

Notice there are now NO rows which contain any null values.

Now let's take a look at the designer column.

In [ ]:
furniture_df['designer'].unique()

Notice there are some designers that are actually descriptions. Since there is too many of these mismatched designers, we will drop the designer column in all (due to time, mostly).

We will also drop the link column as these links are outdated and no longer working.

In [ ]:
furniture_df.drop(columns=['link', 'designer'], inplace=True)
furniture_df

# POKEMON DATAFRAME

We will create a seperate dataframe that contains all the columns relating to pokemon

In [ ]:
pokemon_df = df[['Pokemon', 'Card Type','Generation', 'Card Number', 'Price19']].copy()
pokemon_df.dropna(subset='Pokemon', inplace=True)
pokemon_df.reset_index(inplace=True, drop=True)
pokemon_df = pokemon_df.rename({'Pokemon': 'name', 'Card Type': 'card_type','Generation': 'generation', 'Card Number': 'card_number','Price19': 'price', }, axis=1)
pokemon_df

Let's check for any rows that contain null values in the Pokemon Dataframe.

In [ ]:
pokemon_df[pokemon_df.isnull().any(axis=1)]

Notice there are now NO rows which contain any null values.

# BOOK DATAFRAME

We will create a seperate dataframe that contains all the columns relating to books

In [ ]:
book_df = df[['price5','ranks', 'title', 'no_of_reviews', 'ratings', 'author', 'cover_type', 'year', 'genre' ]].copy()
book_df = book_df.dropna(subset='title')
book_df.reset_index(inplace=True, drop=True)
book_df = book_df.rename({'price5': 'price'}, axis=1)
book_df


Now we will rename some of the columns

Let's check for any rows that contain null values in the Book Dataframe.

In [ ]:
book_df[book_df.isnull().any(axis=1)]

There is one record whose genre is None. We can replace it with the correct data.

In [ ]:
book_df.loc[1253,'genre'] = "Non Fiction"
book_df.loc[[1253]]


Let's check (again) for any rows that contain null values in the Book Dataframe.

In [ ]:
book_df[book_df.isnull().any(axis=1)]

In this dataframe, there are some mismatch values in columns. Lets try to fix them.

Notice the incorrect values in the 'cover type' column: '4.8', 'Dr. Steven R Gundry  MD'

In [ ]:
book_df['cover_type'].unique()

In [ ]:
book_df.loc[book_df['cover_type'] == '4.8']

Almost all of the column values for this record are mismatch. So we must put them in their correct spots

In [ ]:
book_df.loc[289,'title'] = '"Moleskine Classic Notebook, Hard Cover, Large (5"" x 8.25"") Ruled/Lined, Black, 240 Pages"'
book_df.loc[289,'no_of_reviews'] = '22,268'
book_df.loc[289,'ratings'] = '4.8'
book_df.loc[289,'author'] = "Moleskine Store"
book_df.loc[289,'cover_type'] = "Hardcover"
book_df.loc[289,'year'] = "2018"
book_df.loc[289, 'genre'] = 'Non Fiction'
book_df.loc[[289]]

In [ ]:
book_df.loc[book_df['cover_type'] == 'Dr. Steven R Gundry  MD']

After some research, index: 906 is the incorrect duplicate of index: 867. Therefore we can drop it.

In [ ]:
book_df.drop([906], inplace=True)
book_df.loc[book_df['cover_type'] == 'Dr. Steven R Gundry  MD']

Now we can fix the mismatching with this row.

In [ ]:
book_df.loc[867,'title'] = '"The Plant Paradox: The Hidden Dangers in ""Healthy"" Foods That Cause Disease and Weight Gain (The Plant Paradox, 1)"'
book_df.loc[867,'no_of_reviews'] = '14,283'
book_df.loc[867,'ratings'] = '4.4'
book_df.loc[867,'author'] = "Dr. Steven R Gundry MD"
book_df.loc[867,'cover_type'] = "Hardcover"
book_df.loc[867,'year'] = "2017"
book_df.loc[867, 'genre'] = 'Non Fiction'
book_df.loc[[867]]

All the cover types should be fixed now, and we also got to clean up a lot of mismatched data that was revealed in other columns as well!

In [ ]:
book_df['cover_type'].unique()

Let's take a look at the genre column now.

In [ ]:
book_df['genre'].unique()

As we can see there is at least one row whose genre is unknown so let's fix that.

In [ ]:
# Locate the row with the unknown value in the genre column
book_df.loc[book_df['genre'] == 'unknown']

# # reassign the value to Non Fiction
book_df.loc[713,'genre'] = "Non Fiction"

# Check to see new genre
book_df.iloc[[713]]

Now the genre column should only contain 'Fiction' and 'Non Fiction' as values

In [ ]:
book_df['genre'].unique()

# FOOD DATAFRAME

We will create a seperate dataframe that contains all the columns relating to food.

In [ ]:
food_df = df[['price5', 'date', 'product', 'market', 'size']].copy()
food_df = food_df.dropna(subset= ('product', 'market', 'price5'))
food_df.reset_index(inplace=True, drop=True)
food_df = food_df.rename({'price5': 'price', 'size': 'measurement'}, axis=1)
food_df

Let's check for any rows that contain null values in the Food Dataframe.

In [ ]:
food_df[food_df.isnull().any(axis=1)]

Notice there are now NO rows which contain any null values.

Let's take a look at the measurement column of the Food Database, and look at the unique values its holds.

In [ ]:
food_df['measurement'].unique()

Notice there is '1 liter' and 'liter' which are practically the same thing. So we will change '1 liter' to be 'liter'.

We will also change '300g' to be 'gram' and 'kg' to be 'kilogram' to keep the measurement naming conventions consistent.

In [ ]:
food_df['measurement'] = np.where((food_df['measurement'] == '1 liter'), "liter", food_df['measurement'])
food_df['measurement'] = np.where((food_df['measurement'] == '300 g'), "gram", food_df['measurement'])
food_df['measurement'] = np.where((food_df['measurement'] == 'kg'), "kilogram", food_df['measurement'])


Now we can see that all the measurement names are consistent and there are no duplicates.

In [ ]:
food_df['measurement'].unique()

# LOAD

Now that we have our clean dataframes, we can load them into their own csv files.

In [ ]:
# Will create new csv files in 'csv_files' folder if they do not already exist
furniture_df.to_csv('csv_files/furniture.csv', index=False) 
pokemon_df.to_csv('csv_files/pokemon.csv', index=False)
book_df.to_csv('csv_files/book.csv', index=False)
food_df.to_csv('csv_files/food.csv', index=False)